## Device use T4 * 2

#### Clone WMamba & U-Mamba Repo

In [ ]:
!git clone https://github.com/Yui-Arthur/WMamba.git
%cd WMamba
!git clone https://github.com/bowang-lab/U-Mamba.git

#### Mabma & U-Mamba & WMamba reqirement

In [ ]:
!pip install causal-conv1d>=1.2.0 --no-cache-dir > /dev/null
!pip install mamba-ssm --no-cache-dir > /dev/null
!cd U-Mamba/umamba && (pip install -e .) > /dev/null

In [ ]:
!pip install ptwt >  /dev/null
!pip install PyWavelets >  /dev/null

In [ ]:
!sudo bash setup_wmamba.sh

### set env variable

In [ ]:
### DO NOT EDIT !!! ###
nnUNet_preprocessed="/home/nnUNet_preprocessed/"
nnUNet_raw="/home/nnUNet_raw/"
nnUNet_results="/kaggle/working/"

%env nnUNet_preprocessed={nnUNet_preprocessed}
%env nnUNet_raw={nnUNet_raw}
%env nnUNet_results={nnUNet_results}



### parameter (can modify)

In [ ]:
### Epoch ###
# **the maximum epoch include all train epoch in every session** #
%env training_epochs=2

### Checkpoint ###
# every {checkpoint_save_every} epochs will save one checkpoint
%env checkpoint_save_every = 10
load_checkpoint = True
prev_checkpoint_f0 = ""
prev_checkpoint_f1 = ""

### Kaggle Dataset Input Path ###
dataset_path = "/kaggle/input/covid-19/Task04_Hippocampus.mp4"
### Dataset ID ###
dataset_id = 4

### Set Trainer/Model ###
trainer = "nnUNetTrainerWMambaBot"


### dataset prepaired

In [ ]:
if dataset_id == 110:
    """===== Dataset 110 Covid-19 ===="""
    """ unrar data folder and move to home. """
    !apt install unrar
    !(unrar x {dataset_path} /home/nnUNet_raw/) >  /dev/null
    """dataset preoproccess about 20 mins."""
    !nnUNetv2_plan_and_preprocess -d 110 --verify_dataset_integrity -c 3d_fullres
    dataset_id = 110
    dataset_name = "Dataset110_CTImage"
else:
    """===== Dastaset 4 Hippocampus ===="""
    """ unzip data folder and move to home. """
    !(tar -C  /home/ -xvf {dataset_path}) > /dev/null
    """ dataset preoproccess. """
    !nnUNetv2_convert_MSD_dataset -i /home/Task04_Hippocampus
    !nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity -c 3d_fullres
    dataset_id = 4
    dataset_name = "Dataset004_Hippocampus"


### train

In [ ]:
from pathlib import Path
if not load_checkpoint:
    ### first training (no checkpoint) ###
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --npz
else:
    ### load checkpoint, continue train
    current_trainer_folder = f"{nnUNet_results}{dataset_name}/{trainer}__nnUNetPlans__3d_fullres/"

    !mkdir -p {current_trainer_folder}/fold_0
    !mkdir -p {current_trainer_folder}/fold_1

    !cp {prev_checkpoint_f0} {current_trainer_folder}/fold_0
    !cp {prev_checkpoint_f1} {current_trainer_folder}/fold_1
    
    # check the checkpoint can be successful loaded
    f0 = [i.name for i in Path(current_trainer_folder+"fold_0").glob("*.pth")]
    f1 = [i.name for i in Path(current_trainer_folder+"fold_1").glob("*.pth")]
    print("Load Checkpoint fold 0",f0)
    print("Load Checkpoint fold 1",f1)
    if len(f0) <1 or len(f1) <1:
        raise ValueError("Load Checkpoint Failed")
    
    !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --c --npz & \
    CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --c  --npz

### find best fold

In [ ]:
!nnUNetv2_find_best_configuration {dataset_id} -c 3d_fullres -tr nnUNetTrainerWMambaBot -f 0 1 > /dev/null

### predict 

In [ ]:
!nnUNetv2_predict -i "{nnUNet_raw}{dataset_name}/imagesTs"\
-o "{nnUNet_results}{dataset_name}/predTs" \
-d {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1 > /dev/null

In [ ]:
####
!python ./createSubmissionFile.py -s "{nnUNet_results}{dataset_name}/predTs" -d "{nnUNet_results}/predTs_rename"
####
!tar zcvf pred.tar.gz "{nnUNet_results}/predTs_rename"